In [1]:
%matplotlib inline

import numpy as np

import tensorflow.data as tf_data
import tensorflow.keras as keras
import tensorflow.keras.regularizers as regularizers
import tensorflow.train as train
import tensorflow.keras.layers as layers

import matplotlib.pyplot as plt

In [2]:
def load_data(file):
    data = np.loadtxt(file, delimiter=',', skiprows=1)
    return data

def split_train_eval(data, ratio):
    rows = data.shape[0]
    rows = int(rows * ratio)
    train_data, eval_data = np.split(data, [rows,], axis=0)
    
    return train_data, eval_data

def split_x_y(data):
    x = data[:, 1:]/256
    y = data[:, :1]
    
    print(x.shape)
    print(y.shape)
    
    return x, y
    

In [3]:
def build_model():
    model = keras.Sequential()
    
    model.add(layers.Reshape((28, 28, 1), input_shape=(784, )))
    model.add(layers.Conv2D(filters=25, kernel_size=5, activation='relu', input_shape=(28, 28, 1)))
    model.add(layers.Flatten())
    model.add(layers.Dense(10, activation='softmax'))
    
    model.compile(
        optimizer=train.AdamOptimizer(0.001, ),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    
    return model


In [11]:
class ModelEnsemble(object):
    def __init__(self, *models):
        self._models = models
    
    @staticmethod
    def _predict(models, x):
        for datum in x:
            datum = datum.reshape(1, -1)
            
            prediction = [
                model.predict(datum)
                for model in models
            ]

            prediction = np.average(prediction, axis=0)            
            prediction = np.argmax(prediction, axis=1)
            
            yield prediction
            
    def evaluate(self, x, y):
        score = 0

        for prediction, label in zip(
            self._predict(self._models, x),
            y
        ):
            if prediction != label:
                continue
                
            score += 1
        
        return score / x.shape[0]

In [5]:
def visualize_prediction(datum, predictions):
    label = int(datum[0])
    datum = datum[1:]
    
    image = datum.reshape(28, 28)
    
    prediction = np.argmax(predictions)
    
    plt.title("Label: {}, Prediction: {}".format(label, prediction))
    plt.imshow(image, cmap='gray')
    plt.show()

In [6]:
data = load_data('train.csv')


In [7]:
train_data, eval_data = split_train_eval(data, 0.8)

train_x, train_y = split_x_y(train_data)
eval_x, eval_y = split_x_y(eval_data)


(33600, 784)
(33600, 1)
(8400, 784)
(8400, 1)


In [15]:
print('Training model 1')
model_1 = build_model()
model_1.fit(x=train_x, y=train_y, batch_size=100, epochs=10, validation_data=[eval_x, eval_y], shuffle=True, verbose=0)

print('Training model 2')
model_2 = build_model()
model_2.fit(x=train_x, y=train_y, batch_size=100, epochs=10, validation_data=[eval_x, eval_y], shuffle=True, verbose=0)

print('Training model 3')
model_3 = build_model()
model_3.fit(x=train_x, y=train_y, batch_size=100, epochs=10, validation_data=[eval_x, eval_y], shuffle=True, verbose=0)

Training model 1
Training model 2
Training model 3


In [13]:
ensemble = ModelEnsemble(model_1, model_2, model_3)

In [14]:
print('Model 1 validation score:')
model_1.evaluate(eval_x, eval_y)

print('Model 2 validation score:')
model_2.evaluate(eval_x, eval_y)

print('Model 3 validation score:')
model_3.evaluate(eval_x, eval_y)

print('Ensemble validation score:')
ensemble.evaluate(eval_x, eval_y)

Model 1 validation score:
8400/8400 [==============================] - 1s 89us/sample - loss: 0.0658 - acc: 0.9818
Model 2 validation score:
8400/8400 [==============================] - 1s 87us/sample - loss: 0.0782 - acc: 0.9802
Model 3 validation score:
8400/8400 [==============================] - 1s 90us/sample - loss: 0.0706 - acc: 0.9804
Ensemble validation score:


0.9833333333333333